In [1]:
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource

Use sample data from detrend.txt

In [2]:
raw_data = []
with open("denoised_func.txt") as f:
    for line in f.readlines():
        data = line.strip("\n").split(" ")
        data = [float(i) for i in data]
        raw_data.append(data)
        
fc = np.asarray(raw_data).reshape((179,179))

Since we don't have node info just make them up for now

In [3]:
nodes = ["roi%s" % (i+1) for i in range(len(fc))]

In [29]:
xname = []
yname = []
color = []
hcolor = []
alpha = []
for i, node1 in enumerate(nodes):
    for j, node2 in enumerate(nodes):
        xname.append(node1)
        yname.append(node2)
        alpha.append(min(abs(fc[i,j]), 0.9) + 0.1)

        if fc[i,j] < -0.2:
            color.append("#eb5e60")
            hcolor.append("#e31a1c")
        elif fc[i,j] > 0.2:
            color.append("#8fbbd9")
            hcolor.append("#1f78b4")
        else:
            color.append('lightgrey')
            hcolor.append('lightgrey')

source = ColumnDataSource(data=dict(
        xname=xname,
        yname=yname,
        colors=color,
        hcolors=hcolor,
        alphas=alpha,
        count=fc.flatten(),
))

In [30]:
print ("val: %s, color: %s, hcolor: %s"%(fc[0,1],color[1],hcolor[1]))


val: -0.117696130636, color: lightgrey, hcolor: lightgrey


In [41]:
p = figure(title="Functional Connectivity Matrix",
           x_axis_location="above", tools="resize,box_zoom,hover,save,reset",
           x_range=list(reversed(nodes)), y_range=nodes)

In [42]:
p.plot_width = 800
p.plot_height = 800
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

In [43]:
rc = p.rect('xname', 'yname', 0.9, 0.9, source=source,
        color='colors', alpha='alphas', line_color=None, 
        hover_line_color="black", hover_color='hcolors', hover_alpha=1.0)

p.select_one(HoverTool).tooltips = [('names', '@yname, @xname'),
                                  ('FC strength', '@count'),]

In [44]:
output_file("fc_test.html", title="FC Matrix example")

show(p) # show the plot